<a href="https://colab.research.google.com/github/superbaobei/studyFiles/blob/master/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install the Package
Here we're installing it directly from GitHub while it's in development.

In [1]:
!pip install 'vanna[flask,anthropic] @ git+https://github.com/vanna-ai/vanna.git@v2'

  Cloning https://github.com/vanna-ai/vanna.git (to revision v2) to /tmp/pip-install-sveb4maj/vanna_58b8bd257033488a84c59820a6b17875
  Running command git clone --filter=blob:none --quiet https://github.com/vanna-ai/vanna.git /tmp/pip-install-sveb4maj/vanna_58b8bd257033488a84c59820a6b17875
  Resolved https://github.com/vanna-ai/vanna.git to commit 17eb03023cac8b03e4879f7e7cfbca8da69a81bd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 8.2 MB/s eta 0:00:00
  Created wheel for vanna: filename=vanna-2.0.0-py3-none-any.whl size=486183 sha256=c2283150f592c6040a55c61b0e15ed5563f9398edd1c578d8e28e75ee48c0e24
  Stored in directory: /tmp/pip-ephem-wheel-cache-91pd7bp9/wheels/a0/03/47/faeb7cd3837e81c9d7ca63d5b028041f6ef48a5605a94daefd
Successfully built vanna


# Download a Sample Database

In [2]:
import httpx

with open("Chinook.sqlite", "wb") as f:
    with httpx.stream("GET", "https://vanna.ai/Chinook.sqlite") as response:
        for chunk in response.iter_bytes():
            f.write(chunk)

# Imports

In [3]:
from vanna import Agent, AgentConfig
from vanna.servers.fastapi import VannaFastAPIServer
from vanna.core.registry import ToolRegistry
from vanna.core.user import UserResolver, User, RequestContext
from vanna.integrations.anthropic import AnthropicLlmService
from vanna.tools import RunSqlTool, VisualizeDataTool
from vanna.integrations.sqlite import SqliteRunner
from vanna.tools.agent_memory import SaveQuestionToolArgsTool, SearchSavedCorrectToolUsesTool
from vanna.integrations.local.agent_memory import DemoAgentMemory
from vanna.capabilities.sql_runner import RunSqlToolArgs
from vanna.tools.visualize_data import VisualizeDataArgs

# Define your User Authentication
Here we're going to say that if you're logged in as `admin@example.com` then you're in the `admin` group, otherwise you're in the `user` group

In [4]:
class SimpleUserResolver(UserResolver):
    async def resolve_user(self, request_context: RequestContext) -> User:
        # In production, validate cookies/JWTs here
        user_email = request_context.get_cookie('vanna_email')
        if not user_email:
            raise ValueError("Missing 'vanna_email' cookie for user identification")

        print(f"Resolving user for email: {user_email}")

        if user_email == "admin@example.com":
            return User(id="admin1", email=user_email, group_memberships=['admin'])

        return User(id="user1", email=user_email, group_memberships=['user'])

# Define the Tools and Access Control

In [7]:
tools = ToolRegistry()
tools.register_local_tool(RunSqlTool(sql_runner=SqliteRunner(database_path="./Chinook.sqlite")), access_groups=['admin', 'user'])
tools.register_local_tool(VisualizeDataTool(), access_groups=['admin', 'user'])
agent_memory = DemoAgentMemory(max_items=1000)
tools.register_local_tool(SaveQuestionToolArgsTool(), access_groups=['admin'])
tools.register_local_tool(SearchSavedCorrectToolUsesTool(), access_groups=['admin', 'user'])

In [ ]:
# Set up LLM
llm = AnthropicLlmService(model="claude-sonnet-4-5", api_key="sk-ant-...")

# Create agent with your options
agent = Agent(
    llm_service=llm,
    tool_registry=tools,
    user_resolver=SimpleUserResolver(),
    config=AgentConfig(),
    agent_memory=agent_memory

)

# 4. Create and run server
server = VannaFastAPIServer(agent)
server.run()

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

Your app is running at:
https://8000-m-s-25dfkq8fis6a2-d.us-east1-0.prod.colab.dev


INFO:     Started server process [331]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
